<h1>Primo Progetto<h1>

In [63]:
# Importo le librerie richieste
import pandas as pd
from serpapi import GoogleScholarSearch
import json

In [64]:
GoogleScholarSearch.SERP_API_KEY = "5ad7613cf550e6717a98fe89a1313fd076c01bf67a9ff629de82129283fd92cf"

#### 1 | Utilizzo della libreria Python SerpAPI per scaricare, per ciascuno dei 7 autori elencati nel file nodes.csv

In [65]:
# Importo il file nodes.csv
df = pd.read_csv('data/nodes.csv')
display(df)

,name,affiliations,author_id,cited_by,interests
0,David La Barbera,Università Degli Studi Di Udine,MvN8wKEAAAAJ,138,Computer Science
1,Michael Soprano,Postdoctoral Research Fellow at the University...,ocK0qRUAAAAJ,208,Computer Science
2,Kevin Roitero,University of Udine,1xd52jMAAAAJ,688,Artificial Intelligence
3,Stefano Mizzaro,Full professor of Computer Science and Informa...,2wvJC6IAAAAJ,4735,Information retrieval
4,Damiano Spina,"School of Computing Technologies, RMIT University",sLzYrNYAAAAJ,2481,Information Retrieval
5,Gianluca Demartini,Associate Professor at the University of Queen...,PCAiILsAAAAJ,5342,Information Retrieval
6,Eddy Maddalena,"Università degli Studi di Udine, Italy",egHDpFkAAAAJ,553,Information Retrieval


In [61]:

updated_df = pd.DataFrame(columns=['author_id', 'cited_by', 'interests'])

# Itero su ogni autore del dataframe
for index, row in df.iterrows():
    author_name = row['name']
        
    params = {
        "engine" : "google_scholar_profiles",
        "hl": "en",
        "mauthors": author_name
    }
    search = GoogleScholarSearch(params)
    results = search.get_dict()["profiles"]
    
    for index in range(len(results)):
        new_row = None
        if results[index]['name'] == author_name:
            # Estraggo le informazioni che mi interessano
            author_id = results[index]['author_id']
            cited_by = results[index]['cited_by']
            interests = results[index]['interests']
            interests = interests[0]['title']
    
            # Aggiungo una nuova riga al dataframe updated_df
            new_row = {'author_id': author_id, 'cited_by': cited_by, 'interests': interests}
            updated_df = pd.concat([updated_df, pd.DataFrame([new_row])], ignore_index=True)

df_new = pd.merge(df, updated_df, left_index=True, right_index=True)
display(df_new)



,name,affiliations,author_id,cited_by,interests
0,David La Barbera,Università Degli Studi Di Udine,MvN8wKEAAAAJ,138,Computer Science
1,Michael Soprano,Postdoctoral Research Fellow at the University...,ocK0qRUAAAAJ,208,Computer Science
2,Kevin Roitero,University of Udine,1xd52jMAAAAJ,688,Artificial Intelligence
3,Stefano Mizzaro,Full professor of Computer Science and Informa...,2wvJC6IAAAAJ,4735,Information retrieval
4,Damiano Spina,"School of Computing Technologies, RMIT University",sLzYrNYAAAAJ,2481,Information Retrieval
5,Gianluca Demartini,Associate Professor at the University of Queen...,PCAiILsAAAAJ,5342,Information Retrieval
6,Eddy Maddalena,"Università degli Studi di Udine, Italy",egHDpFkAAAAJ,553,Information Retrieval


In [62]:
# Sovrascrivo il file nodes.csv con le nuove informazioni trovate
df_new.to_csv('data/nodes.csv', index=False)

#### 2 | Per ogni autore utilizzo il suo ID per accedere al relativo profilo GoogleScholar e scaricare l’elenco dei suoi coautori, sempre via SerpAPI

In [89]:
import pandas as pd
from serpapi import GoogleScholarSearch

# KEY
GoogleScholarSearch.SERP_API_KEY = "2aa87b49a5155f85c38fc508e5ca126093254610c745dc225631ef7e9baf47cd"

def get_coauthors(author_id):
    """Ottiene l'elenco degli ID dei coautori per un dato author_id."""
    try:
        params = {
            "engine": "google_scholar_author",
            "author_id": author_id,
            "hl": "en"
        }
        search = GoogleScholarSearch(params)
        results = search.get_dict()
        co_authors = results.get("co_authors", [])
        return [coauthor["author_id"] for coauthor in co_authors]
    except Exception as e:
        print(f"Errore nella ricerca dei coautori per {author_id}: {e}")
        return []

def get_author_details_by_id(author_id):
    """Ottiene i dettagli di un autore dato il suo ID."""
    try:
        params = {
            "engine": "google_scholar_author",
            "author_id": author_id,
            "hl": "en"
        }
        search = GoogleScholarSearch(params)
        results = search.get_dict()
        author = results.get("author", {})
        return {
            "name": author.get("name"),
            "affiliations": author.get("affiliations"),
            "author_id": author_id,
            "cited_by": author.get("cited_by"),
            "interests": ",".join([interest["title"] for interest in author.get("interests", [])])
        }
    except Exception as e:
        print(f"Errore nella ricerca dei dettagli per {author_id}: {e}")
        return None

# Carica il DataFrame degli autori originari
df_new = pd.read_csv('data/nodes.csv')

# DataFrame per i dettagli dei coautori
coauthors_df = pd.DataFrame(columns=['name', 'affiliations', 'author_id', 'cited_by', 'interests'])

# Ciclo for per popolare coauthors_df con le due funzioni
for author_id in df_new['author_id']:
    coauthor_ids = get_coauthors(author_id)
    print(f"Coautori per {author_id}: {coauthor_ids}")

    for coauthor_id in coauthor_ids:
        details = get_author_details_by_id(coauthor_id)
        print(f"Dettagli per {coauthor_id}: {details}")

        if details:
            coauthors_df = pd.concat([coauthors_df, pd.DataFrame([details])], ignore_index=True)


# Rimuove le righe duplicate basandosi sull'`author_id`
coauthors_df = coauthors_df.drop_duplicates(subset='author_id')

# Salvataggio del nuovo DataFrame
coauthors_df.to_csv('data/coauthors.csv', index=False)

Coautori per MvN8wKEAAAAJ: ['1xd52jMAAAAJ', '2wvJC6IAAAAJ', 'PCAiILsAAAAJ', 'sLzYrNYAAAAJ', 'ocK0qRUAAAAJ', 'BF3B4KAAAAAJ', 'y8Jy518AAAAJ', 'UTMHOZMAAAAJ', '3jQ1Zh4AAAAJ']
Dettagli per 1xd52jMAAAAJ: {'name': 'Kevin Roitero', 'affiliations': 'University of Udine', 'author_id': '1xd52jMAAAAJ', 'cited_by': None, 'interests': 'Artificial Intelligence,Crowdsourcing and Human Computation,Information Retrieval'}
Dettagli per 2wvJC6IAAAAJ: {'name': 'Stefano Mizzaro', 'affiliations': 'Full professor of Computer Science and Information Technology', 'author_id': '2wvJC6IAAAAJ', 'cited_by': None, 'interests': 'Information retrieval,mobile systems,scholarly publishing,crowdsourcing,misinformation and fake news detection'}
Dettagli per PCAiILsAAAAJ: {'name': 'Gianluca Demartini', 'affiliations': 'Associate Professor at the University of Queensland', 'author_id': 'PCAiILsAAAAJ', 'cited_by': None, 'interests': 'Information Retrieval,Semantic Web,Human Computation,Crowdsourcing'}
Dettagli per sLzYrNYAA